In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv("backtest_output/metrics/SUMMARY_BTCUSDT_20251005_073131.csv")

num_cols = [
    "rows",
    "accuracy",
    "precision",
    "recall",
    "f1",
    "auc",
    "best_threshold",
    "trades",
    "hit_rate",
    "avg_net_ret_per_bar",
    "avg_net_ret_per_trade",
    "total_net_return",
    "sharpe_like",
    "cost_roundtrip",
    "r2",
    "mae_bps",
    "mape_bps",
]
for c in num_cols:
    if c not in df.columns:
        df[c] = np.nan
    df[c] = pd.to_numeric(df[c], errors="coerce")

df["task"] = np.where(df["r2"].notna(), "regress", "classify")
df.head()

,symbol,interval,start_str,timelag,model,rows,split_mode,test_size,class_weight,label_mode,...,hit_rate,avg_net_ret_per_bar,avg_net_ret_per_trade,total_net_return,sharpe_like,cost_roundtrip,r2,mae_bps,mape_bps,task
0,BTCUSDT,4h,120 days ago UTC,16,linreg,673,time,0.2,NaN,return_bps,...,NaN,15.131373,NaN,105.919608,0.326890,0.0026,-0.253622,46.983941,NaN,regress
1,BTCUSDT,4h,120 days ago UTC,16,linreg,673,time,0.2,NaN,return_bps,...,NaN,14.641549,NaN,102.490840,0.320304,0.0026,-0.255178,46.958354,NaN,regress
2,BTCUSDT,4h,120 days ago UTC,16,rf_reg,673,time,0.2,NaN,return_bps,...,NaN,-15.769174,NaN,-315.383487,-0.349751,0.0026,-1.278851,65.041553,NaN,regress
3,BTCUSDT,4h,120 days ago UTC,16,hgb_reg,673,time,0.2,NaN,return_bps,...,NaN,-1.776512,NaN,-8.882558,-0.039016,0.0026,-0.725038,57.026228,NaN,regress
4,BTCUSDT,4h,120 days ago UTC,16,svr,673,time,0.2,NaN,return_bps,...,NaN,NaN,NaN,0.000000,NaN,0.0026,-0.001662,38.437100,NaN,regress


In [10]:
MIN_ROWS = 200  # drop tiny runs
df1 = df.copy()

df1 = df1.loc[df1["rows"] >= MIN_ROWS]
df1 = df1.loc[df1["trades"].fillna(0) > 0]
df1 = df1.loc[~((df1["task"] == "regress") & (df1["r2"].isna()))]

print(f"After sanity filters: {len(df)} -> {len(df1)} rows")
df1.head()

After sanity filters: 99 -> 77 rows


,symbol,interval,start_str,timelag,model,rows,split_mode,test_size,class_weight,label_mode,...,hit_rate,avg_net_ret_per_bar,avg_net_ret_per_trade,total_net_return,sharpe_like,cost_roundtrip,r2,mae_bps,mape_bps,task
0,BTCUSDT,4h,120 days ago UTC,16,linreg,673,time,0.2,NaN,return_bps,...,NaN,15.131373,NaN,105.919608,0.326890,0.0026,-0.253622,46.983941,NaN,regress
1,BTCUSDT,4h,120 days ago UTC,16,linreg,673,time,0.2,NaN,return_bps,...,NaN,14.641549,NaN,102.490840,0.320304,0.0026,-0.255178,46.958354,NaN,regress
2,BTCUSDT,4h,120 days ago UTC,16,rf_reg,673,time,0.2,NaN,return_bps,...,NaN,-15.769174,NaN,-315.383487,-0.349751,0.0026,-1.278851,65.041553,NaN,regress
3,BTCUSDT,4h,120 days ago UTC,16,hgb_reg,673,time,0.2,NaN,return_bps,...,NaN,-1.776512,NaN,-8.882558,-0.039016,0.0026,-0.725038,57.026228,NaN,regress
5,BTCUSDT,4h,120 days ago UTC,16,hgb_reg,673,time,0.2,NaN,return_bps,...,NaN,-1.776512,NaN,-8.882558,-0.039016,0.0026,-0.725332,57.011342,NaN,regress


In [11]:
MIN_SHARPE = 0.3
df2 = df1.copy()

df2 = df2.loc[df2["total_net_return"] > 0]
df2 = df2.loc[df2["avg_net_ret_per_bar"] > 0]
df2 = df2.loc[df2["sharpe_like"] > MIN_SHARPE]

print(f"After econ filters: {len(df1)} -> {len(df2)} rows")
df2.head()

After econ filters: 77 -> 25 rows


,symbol,interval,start_str,timelag,model,rows,split_mode,test_size,class_weight,label_mode,...,hit_rate,avg_net_ret_per_bar,avg_net_ret_per_trade,total_net_return,sharpe_like,cost_roundtrip,r2,mae_bps,mape_bps,task
0,BTCUSDT,4h,120 days ago UTC,16,linreg,673,time,0.2,NaN,return_bps,...,NaN,15.131373,NaN,105.919608,3.268899e-01,0.0026,-0.253622,46.983941,NaN,regress
1,BTCUSDT,4h,120 days ago UTC,16,linreg,673,time,0.2,NaN,return_bps,...,NaN,14.641549,NaN,102.490840,3.203045e-01,0.0026,-0.255178,46.958354,NaN,regress
13,BTCUSDT,8h,120 days ago UTC,16,rf_reg,313,time,0.2,NaN,return_bps,...,NaN,27.535308,NaN,302.888386,3.326425e-01,0.0026,-0.229406,69.070436,NaN,regress
14,BTCUSDT,8h,120 days ago UTC,16,hgb_reg,313,time,0.2,NaN,return_bps,...,NaN,3.073171,NaN,3.073171,3.073171e+09,0.0026,-0.349161,70.283799,NaN,regress
16,BTCUSDT,8h,120 days ago UTC,16,hgb_reg,313,time,0.2,NaN,return_bps,...,NaN,3.073171,NaN,3.073171,3.073171e+09,0.0026,-0.349161,70.283786,NaN,regress


In [21]:
# --- 3) Regression-specific filters ----------------------------------------
df3 = df2.copy()
is_reg = df3["task"] == "regress"

# Drop bad regressors
df3 = df3.loc[~(is_reg & (df3["r2"] <= -0.5))]

# Drop huge MAE outliers (3× median per interval)
for iv, sub in df3.groupby("interval"):
    med = np.nanmedian(sub["mae_bps"])
    if not np.isnan(med) and med > 0:
        bad = (df3["interval"] == iv) & (df3["mae_bps"] > 3 * med)
        df3 = df3.loc[~bad]

print(f"After regression quality filters: {len(df2)} -> {len(df3)} rows")
df3.head()

After regression quality filters: 25 -> 18 rows


,symbol,interval,start_str,timelag,model,rows,split_mode,test_size,class_weight,label_mode,...,hit_rate,avg_net_ret_per_bar,avg_net_ret_per_trade,total_net_return,sharpe_like,cost_roundtrip,r2,mae_bps,mape_bps,task
0,BTCUSDT,4h,120 days ago UTC,16,linreg,673,time,0.2,NaN,return_bps,...,NaN,15.131373,NaN,105.919608,3.268899e-01,0.0026,-0.253622,46.983941,NaN,regress
1,BTCUSDT,4h,120 days ago UTC,16,linreg,673,time,0.2,NaN,return_bps,...,NaN,14.641549,NaN,102.490840,3.203045e-01,0.0026,-0.255178,46.958354,NaN,regress
13,BTCUSDT,8h,120 days ago UTC,16,rf_reg,313,time,0.2,NaN,return_bps,...,NaN,27.535308,NaN,302.888386,3.326425e-01,0.0026,-0.229406,69.070436,NaN,regress
14,BTCUSDT,8h,120 days ago UTC,16,hgb_reg,313,time,0.2,NaN,return_bps,...,NaN,3.073171,NaN,3.073171,3.073171e+09,0.0026,-0.349161,70.283799,NaN,regress
16,BTCUSDT,8h,120 days ago UTC,16,hgb_reg,313,time,0.2,NaN,return_bps,...,NaN,3.073171,NaN,3.073171,3.073171e+09,0.0026,-0.349161,70.283786,NaN,regress


In [22]:
df4 = df3.copy()

stab = df4.groupby(["interval", "model"], as_index=False).agg(
    n_runs=("sharpe_like", "size"),
    median_sharpe=("sharpe_like", "median"),
    hit_med=("hit_rate", "median"),
    net_med=("total_net_return", "median"),
)

stab_keep = stab.loc[
    (stab["median_sharpe"] > 0) & (stab["net_med"] > 0), ["interval", "model"]
]
df5 = df4.merge(stab_keep, on=["interval", "model"], how="inner")

print(f"After stability filter: {len(df4)} -> {len(df5)} rows")
stab.sort_values(["interval", "median_sharpe"], ascending=[True, False]).head(10)

After stability filter: 18 -> 18 rows


,interval,model,n_runs,median_sharpe,hit_med,net_med
0,1d,hgb_reg,7,6.768291e-01,NaN,770.793118
1,4h,linreg,2,3.235972e-01,NaN,104.205224
3,8h,rf_reg,2,8.260318e+10,NaN,234.047374
2,8h,hgb_reg,7,3.073171e+09,NaN,3.073171


In [ ]:
# --- 6) Minimum activity & cost sanity ----------------
df6 = df5.copy()

# 1) Enforce minimum trades
MIN_TRADES = 5
df6 = df6.loc[(df6["trades"].fillna(0).astype(int) >= MIN_TRADES)]

# Prefer explicit avg_net_ret_per_trade if present & looks like bps already.
avg_trade_bps = df6.get("avg_net_ret_per_trade")

# If avg_net_ret_per_trade is missing or mostly NaN, derive from totals
if (avg_trade_bps is None) or (avg_trade_bps.isna().mean() > 0.3):
    avg_trade_bps = pd.Series(np.nan, index=df6.index)

# Fill from total / trades where possible (avoid divide-by-zero)
has_trades = df6["trades"] > 0
avg_trade_bps = avg_trade_bps.where(
    ~avg_trade_bps.isna(),
    df6["total_net_return"] / df6["trades"].where(has_trades, np.nan),
)

# 3) Convert cost to bps for fair comparison
# cost_roundtrip is a fraction (e.g., 0.0026 = 26 bps)
cost_bps = df6["cost_roundtrip"] * 10_000.0

# 4) Pick a mild safety margin (e.g., +5%) and apply cost sanity in bps
COST_MULTIPLIER = 1.05
ok_cost = avg_trade_bps >= (COST_MULTIPLIER * cost_bps)

# If avg_trade_bps is still NaN for a row, treat as fail
ok_cost = ok_cost.fillna(False)

before = len(df6)
df6 = df6.loc[ok_cost]

print(f"After activity & cost filter: {before} -> {len(df6)} rows")
df6.head()

After activity & cost filter: 10 -> 8 rows


,symbol,interval,start_str,timelag,model,rows,split_mode,test_size,class_weight,label_mode,...,hit_rate,avg_net_ret_per_bar,avg_net_ret_per_trade,total_net_return,sharpe_like,cost_roundtrip,r2,mae_bps,mape_bps,task
2,BTCUSDT,8h,120 days ago UTC,16,rf_reg,313,time,0.2,NaN,return_bps,...,NaN,27.535308,NaN,302.888386,0.332642,0.0026,-0.229406,69.070436,NaN,regress
11,BTCUSDT,1d,365 days ago UTC,16,hgb_reg,318,time,0.2,NaN,return_bps,...,NaN,77.079312,NaN,770.793118,0.676829,0.0026,-0.353955,159.443596,NaN,regress
12,BTCUSDT,1d,365 days ago UTC,16,hgb_reg,318,time,0.2,NaN,return_bps,...,NaN,77.079312,NaN,770.793118,0.676829,0.0026,-0.353955,159.443596,NaN,regress
13,BTCUSDT,1d,365 days ago UTC,16,hgb_reg,318,time,0.2,NaN,return_bps,...,NaN,77.079312,NaN,770.793118,0.676829,0.0026,-0.353955,159.443583,NaN,regress
14,BTCUSDT,1d,365 days ago UTC,16,hgb_reg,318,time,0.2,NaN,return_bps,...,NaN,77.079312,NaN,770.793118,0.676829,0.0026,-0.353955,159.443596,NaN,regress


In [28]:
# --- 7) Composite score per interval ---------------------------------------

df7 = df6.copy()


def rank_within_interval(g):
    for c in ["sharpe_like", "total_net_return", "avg_net_ret_per_bar", "trades"]:
        g[c + "_z"] = (g[c] - g[c].mean()) / (g[c].std(ddof=0) + 1e-9)
    g = g.merge(
        stab[["interval", "model", "n_runs", "median_sharpe"]],
        on=["interval", "model"],
        how="left",
    )
    g["n_runs"] = g["n_runs"].fillna(1)
    g["med_sharpe_z"] = (g["median_sharpe"] - g["median_sharpe"].mean()) / (
        g["median_sharpe"].std(ddof=0) + 1e-9
    )
    g["score"] = (
        0.45 * g["sharpe_like_z"]
        + 0.25 * g["total_net_return_z"]
        + 0.15 * g["avg_net_ret_per_bar_z"]
        + 0.10 * g["trades_z"]
        + 0.05 * g["med_sharpe_z"]
    )
    return g


df7 = df7.groupby("interval", group_keys=False).apply(rank_within_interval)
df7 = df7.sort_values(["interval", "score"], ascending=[True, False])
df7[
    [
        "interval",
        "model",
        "start_str",
        "task",
        "rows",
        "trades",
        "sharpe_like",
        "total_net_return",
        "avg_net_ret_per_bar",
        "score",
    ]
].head(20)

/tmp/ipykernel_1456351/4038077940.py:21: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df7 = df7.groupby("interval", group_keys=False).apply(rank_within_interval)


,interval,model,start_str,task,rows,trades,sharpe_like,total_net_return,avg_net_ret_per_bar,score
0,1d,hgb_reg,365 days ago UTC,regress,318,10,0.676829,770.793118,77.079312,0.000002
1,1d,hgb_reg,365 days ago UTC,regress,318,10,0.676829,770.793118,77.079312,0.000002
2,1d,hgb_reg,365 days ago UTC,regress,318,10,0.676829,770.793118,77.079312,0.000002
3,1d,hgb_reg,365 days ago UTC,regress,318,10,0.676829,770.793118,77.079312,0.000002
4,1d,hgb_reg,365 days ago UTC,regress,318,10,0.676829,770.793118,77.079312,0.000002
5,1d,hgb_reg,365 days ago UTC,regress,318,10,0.676829,770.793118,77.079312,0.000002
6,1d,hgb_reg,365 days ago UTC,regress,318,10,0.676829,770.793118,77.079312,0.000002
0,8h,rf_reg,120 days ago UTC,regress,313,11,0.332642,302.888386,27.535308,0.000000


In [29]:
# --- 8) Pick winners --------------------------------------------------------
TOPK = 3

winners = (
    df7.sort_values(["interval", "score"], ascending=[True, False])
    .groupby("interval")
    .head(TOPK)
    .reset_index(drop=True)
)

most_stable = (
    df7.sort_values(["interval", "median_sharpe"], ascending=[True, False])
    .groupby("interval")
    .head(1)
    .reset_index(drop=True)
)

print("Top-K by composite score:")
display(
    winners[
        [
            "interval",
            "model",
            "task",
            "start_str",
            "rows",
            "trades",
            "sharpe_like",
            "total_net_return",
            "avg_net_ret_per_bar",
            "score",
        ]
    ]
)

print("\nMost stable per interval (by median Sharpe):")
display(
    most_stable[
        [
            "interval",
            "model",
            "task",
            "rows",
            "trades",
            "median_sharpe",
            "total_net_return",
            "avg_net_ret_per_bar",
        ]
    ]
)

Top-K by composite score:


,interval,model,task,start_str,rows,trades,sharpe_like,total_net_return,avg_net_ret_per_bar,score
0,1d,hgb_reg,regress,365 days ago UTC,318,10,0.676829,770.793118,77.079312,0.000002
1,1d,hgb_reg,regress,365 days ago UTC,318,10,0.676829,770.793118,77.079312,0.000002
2,1d,hgb_reg,regress,365 days ago UTC,318,10,0.676829,770.793118,77.079312,0.000002
3,8h,rf_reg,regress,120 days ago UTC,313,11,0.332642,302.888386,27.535308,0.000000



Most stable per interval (by median Sharpe):


,interval,model,task,rows,trades,median_sharpe,total_net_return,avg_net_ret_per_bar
0,1d,hgb_reg,regress,318,10,6.768291e-01,770.793118,77.079312
1,8h,rf_reg,regress,313,11,8.260318e+10,302.888386,27.535308


In [30]:
# --- 9) Export shortlist ----------------------------------------------------
out_dir = "backtest_output/metrics"
winners.to_csv(f"{out_dir}/SHORTLIST_topk.csv", index=False)
most_stable.to_csv(f"{out_dir}/SHORTLIST_stable.csv", index=False)

print("Saved SHORTLIST_topk.csv and SHORTLIST_stable.csv")

Saved SHORTLIST_topk.csv and SHORTLIST_stable.csv
